<a href="https://colab.research.google.com/github/mariaaapetrovskaya/complingua/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22w2v_hw_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [1]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [2]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Базовые операции с векторами**

In [3]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [4]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Слова, похожие на 'python':
  monty: 0.6886
  php: 0.5865
  perl: 0.5784
  cleese: 0.5447
  flipper: 0.5113


**Первый датасет/модель - fake-news. **
Модель обучена на на датасете фейковых новостей. Размер файлов - 19 MB, содержит текст и метаданные с 244 веб-сайтов (в общей сложности 12 999 сообщений за определенный период в 30 дней). Ее сожно использовать, например, для анализ стиля текстов, для классификации именно фейковыйх новостей и т.д. Ссылка: https://www.kaggle.com/mrisdal/fake-news
**Вторая модель - word2vec-ruscorpora-300.** Ее объем - 198 MB. Она обучена на Русском Национальном Корпусе(около  250M слов). 184973 векторов. Это именно русскоязычная модель, она обучена на разных текстах (художественная литература, новости, научные статьи). Может быть использована для обработки русского текста. Ссылки: https://www.academia.edu/24306935/WebVectors_a_Toolkit_for_Building_Web_Interfaces_for_Vector_Semantic_Models
http://rusvectores.org/en/
**Третья - glove-twitter-100. **
1193514 векторов. Объем файлов - 387 MB. Содержит тексты из Twitterа (2B tweets, 27B tokens, 1.2M vocab, uncased). Компактная модель, она хорошо работает с разными сокращениями, хештегами и сленгом (для анализа подобных текстов ее и можно использовать). Ссылки: https://nlp.stanford.edu/projects/glove/
https://nlp.stanford.edu/pubs/glove.pdf

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [5]:
w2v_model = api.load('glove-twitter-100')
print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 387.1/387.1MB downloaded
Размер словаря: 1193514
Размерность векторов: 100


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [18]:

def find_similar_words(word, topn=10):
    print(f"Наиболее близкие слова для слова '{word}':")

    try:
        similar_words = w2v_model.most_similar(word, topn=topn)
        for similar_word, score in similar_words:
            print(f"  {similar_word}: {score:.3f}")
    except KeyError:
        print(f"  Слово '{word}' не найдено в модели")

# Тестируем функцию с разными типами слов
print("\n" + "="*50)
print("обычные слова")
find_similar_words("apple")

print("\n" + "="*50)
print("сленг")
find_similar_words("lol")

print("\n" + "="*50)
print("хештеги")
find_similar_words("#health")



обычные слова
Наиболее близкие слова для слова 'apple':
  microsoft: 0.823
  samsung: 0.779
  iphone: 0.767
  google: 0.762
  nokia: 0.749
  blackberry: 0.741
  nexus: 0.741
  ipad: 0.737
  galaxy: 0.729
  smartphone: 0.719

сленг
Наиболее близкие слова для слова 'lol':
  lmao: 0.917
  lmfao: 0.870
  yeah: 0.854
  lolol: 0.854
  aha: 0.848
  wtf: 0.847
  thats: 0.845
  dude: 0.843
  but: 0.839
  shit: 0.834

хештеги
Наиболее близкие слова для слова '#health':
  #fitness: 0.704
  #motivation: 0.550
  #weightloss: 0.549
  #shape: 0.537
  #workout: 0.520
  #shopping: 0.498
  #training: 0.474
  наблюдаю: 0.474
  kuranla: 0.474
  hiddet: 0.466


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [19]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [20]:
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    sg=1
)

print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

try:
    similar = model.wv.most_similar('паста', topn=5)
    print("Слова, похожие на 'паста':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'паста' не найдено в словаре")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...
Слова, похожие на 'паста':
  сахар: 0.2804
  горшок: 0.2443
  тушить: 0.2439
  говядина: 0.2105
  суп: 0.1926


4. Проверьте модель

In [22]:

try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [23]:

try:
    similar = model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

try:
    similar = model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142
Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [36]:
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

print(f"Слова в словаре FastText: {list(ft_model.wv.key_to_index.keys())[:10]}...")

Слова в словаре FastText: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [37]:
test_words = ['варить', 'духовка', 'овощи']

for word in test_words:
    print(f"\n{'='*50}")
    print(f"Слова, похожие на '{word}':")

    try:
        similar_words = ft_model.wv.most_similar(word, topn=5)
        for similar_word, score in similar_words:
            print(f"  {similar_word}: {score:.4f}")
    except KeyError:
        print(f"  Слово '{word}' не найдено в модели")


Слова, похожие на 'варить':
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622

Слова, похожие на 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944

Слова, похожие на 'овощи':
  жарить: 0.2960
  фольга: 0.2574
  морковь: 0.2297
  соус: 0.2172
  торт: 0.2094


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [38]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

print("\n" + "="*60)
print("Сравнение Word2Vec и FastText для слов из корпуса:")
print("="*60)

compare_models('варить')
compare_models('духовка')
compare_models('овощи')
compare_models('паста')


Сравнение Word2Vec и FastText для слов из корпуса:

Сравнение для слова: 'варить'
  Word2Vec: слово не найдено
  FastText: ['жарить', 'парить']

Сравнение для слова: 'духовка'
  Word2Vec: слово не найдено
  FastText: ['взбивать', 'лимон']

Сравнение для слова: 'овощи'
  Word2Vec: слово не найдено
  FastText: ['жарить', 'фольга']

Сравнение для слова: 'паста'
  Word2Vec: слово не найдено
  FastText: ['хлеб', 'печь']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [24]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [25]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [26]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [27]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [28]:
compare_documents(2, 4)

Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [30]:
similar_to_1 = doc_model.dv.most_similar("doc_1", topn=len(doc_model.dv.key_to_index))
print("Сортир. по схожести с doc_1:")
for doc_tag, similarity in similar_to_1:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f} - {documents[doc_id]}")

# Самый похожий документ на doc_1
most_similar_doc = similar_to_1[0]
print(f"\nСамый похожий документ на doc_1:")
print(f"  {most_similar_doc[0]}: {most_similar_doc[1]:.4f}")
doc_id = int(most_similar_doc[0].split('_')[1])
print(f"  Текст: {documents[doc_id]}")

Сортир. по схожести с doc_1:
  doc_0: 0.2735 - machine learning is interesting
  doc_3: 0.2031 - artificial intelligence is amazing
  doc_2: -0.0573 - python programming for data science
  doc_4: -0.2546 - computer vision processes images

Самый похожий документ на doc_1:
  doc_0: 0.2735
  Текст: machine learning is interesting


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [35]:
from gensim.models import Word2Vec
import nltk
from nltk.corpus import brown
import string


nltk.download('brown')
nltk.download('punkt')

sentences = brown.sents()

def preprocess_text(sentences, sample_size=10000):
    processed_sentences = []
    translator = str.maketrans('', '', string.punctuation)

    for sentence in sentences[:sample_size]:
        cleaned = [word.lower().translate(translator) for word in sentence]
        words = [word for word in cleaned if word.isalpha() and len(word) > 1]
        if len(words) >= 3:
            processed_sentences.append(words)

    return processed_sentences


def remove_stopwords(sentences):
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    processed_sentences = []

    for sentence in sentences:
        filtered_sentence = [word for word in sentence if word not in stop_words]
        processed_sentences.append(filtered_sentence)

    return processed_sentences


processed_sentences = preprocess_text(sentences, sample_size=55000)
sentences_without_stopwords = remove_stopwords(processed_sentences)


test_words = ['user', 'government', 'congress']
vector_sizes = [10, 50, 100]
models = {}

for vs in vector_sizes:
    print(f"\nобучение модели vector_size={vs}...")
    model = Word2Vec(
        sentences=sentences_without_stopwords,
        vector_size=vs,
        window=3,
        min_count=2,
        workers=4,
        sg=1
    )
    models[vs] = model
    print(f"  Размер словаря: {len(model.wv.key_to_index)}")

print("\n" + "="*60)
print("Сравнение качества моделей:")
print("="*60)

for word in test_words:
    print(f"\nПохожие слова для '{word}':")

    for vs in vector_sizes:
        print(f"\n  vector_size={vs}:")
        try:
            similar_words = models[vs].wv.most_similar(word, topn=3)
            for similar_word, score in similar_words:
                print(f"    {similar_word}: {score:.3f}")
        except KeyError:
            print(f"    Слово '{word}' не найдено в модели")

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



обучение модели vector_size=10...
  Размер словаря: 25967

обучение модели vector_size=50...
  Размер словаря: 25967

обучение модели vector_size=100...
  Размер словаря: 25967

Сравнение качества моделей:

Похожие слова для 'user':

  vector_size=10:
    conformed: 0.999
    bulky: 0.997
    affinity: 0.997

  vector_size=50:
    intentional: 0.997
    traced: 0.997
    validity: 0.996

  vector_size=100:
    phonologic: 0.998
    kohnstammnegative: 0.998
    diagrams: 0.998

Похожие слова для 'government':

  vector_size=10:
    union: 0.977
    court: 0.959
    aid: 0.957

  vector_size=50:
    foreign: 0.963
    union: 0.961
    federal: 0.958

  vector_size=100:
    foreign: 0.977
    federal: 0.976
    union: 0.966

Похожие слова для 'congress':

  vector_size=10:
    court: 0.964
    report: 0.961
    representatives: 0.959

  vector_size=50:
    supreme: 0.970
    constitution: 0.961
    soviet: 0.951

  vector_size=100:
    supreme: 0.972
    america: 0.969
    court: 0.968
